In [1]:
import xmltodict
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ls data

 Volume in drive E has no label.
 Volume Serial Number is 19C7-A2B3

 Directory of E:\metis\ds4_fletcher\data

08/17/2015  12:55 PM    <DIR>          .
08/17/2015  12:55 PM    <DIR>          ..
03/09/2015  06:04 PM        60,201,826 Badges.xml
03/09/2015  06:04 PM       238,716,194 Comments.xml
03/09/2015  06:05 PM     1,144,804,160 PostHistory.xml
03/09/2015  06:06 PM         6,192,845 PostLinks.xml
03/09/2015  06:06 PM       736,314,717 Posts.xml
03/09/2015  06:06 PM           367,147 Tags.xml
03/09/2015  06:06 PM       100,081,558 Users.xml
03/09/2015  06:07 PM       210,233,404 Votes.xml
               8 File(s)  2,496,911,851 bytes
               2 Dir(s)  638,832,103,424 bytes free


In [4]:
tree = ET.parse('data\Tags.xml')
root = tree.getroot()

In [5]:
root.tag

'tags'

In [6]:
tags_df = helper.get_xml_file_as_df('data\Tags.xml',max_num_children=100)

In [7]:
tags_df.head()

,Count,ExcerptPostId,Id,TagName,WikiPostId
0,206,392824,203,firmware,392823
1,993,320795,204,upgrade,320794
2,13643,244736,205,networking,244735
3,3468,257013,206,audio,257012
4,2825,256215,207,video,256214


In [8]:
posts_df = helper.get_xml_file_as_df('data\Posts.xml',max_num_children=1000)

In [9]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,NaN,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,NaN,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN


In [10]:
posts_df.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount'],
      dtype='object')

In [11]:
posts_df['BodyCleaned']=posts_df['Body'].apply(lambda x : helper.strip_tags(x))

In [12]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,BodyCleaned
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584,A Vista virtual machine I use only has a 10 GB...
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654,I used to record important technical meetings ...
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN,You could use a filter on the recording. If I ...
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN,try out something like:\naudible (mac)\nsoundf...
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN,You definitely need some sort of software to f...


In [13]:
len(posts_df)

1001